# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [18]:
#Your code here
import pandas as pd

df = pd.read_csv('homepage_actions.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [46]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [60]:
"""   
    * How many viewers also clicked?
    * Are there any anomalies with the data; did anyone click who didn't view?"""
cids=df[df.action =='click'].id.unique()
vids =df[df.action =='view'].id.unique()
df_click =df[df.action =='click']
df_view = df[df.action =='view']
print("Number of viewrs:{}\tNumber of clickers:{}".format(len(vids), len(cids)))
df_click_view = df_view.set_index('id').join(df_click.set_index('id'),rsuffix='_click', how ='inner')
cvids = df_click_view.reset_index().id.unique()
print('number of people who viewed and clicked:',len(cvids))
print("number of people who viewed and didn't clicked:", len(vids)-len(cvids) )
print("number of people who clicked and didn't viewd:", len(cids)-len(cvids))

Number of viewrs:6328	Number of clickers:1860
number of people who viewed and clicked: 1860
number of people who viewed and didn't clicked: 4468
number of people who clicked and didn't viewd: 0


In [61]:
"""Everyone who clicked viewed the homepage!"""

'Everyone who clicked viewed the homepage!'

In [83]:
"""
    * Is there any overlap between the control and experiment groups? 
    * If so, how do you plan to account for this in your experimental design?
"""
con_df = df[df.group == 'control']
exp_df = df[df.group == 'experiment']
overlap_df = con_df[[x in exp_df.id for x in con_df.id]]
len(overlap_df)
print(f'There is {len(overlap_df)} overlap(s) between control and experiment groups')

There is 0 overlap(s) between control and experiment groups


In [91]:
"Solution "
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [93]:
df['count']=1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [95]:
control = df[df.group =='control'].pivot(index='id', columns ='action', values='count')
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,NaN,1.0
183248,1.0,1.0
183515,NaN,1.0
183524,NaN,1.0


In [96]:
control = control.fillna(value = 0)

In [97]:
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [98]:
print(f"Sample sizes:\tControl: {len(control)} \tExperiment: {len(experiment)}")
print(f"Total Clicks:\tControl:{control.click.sum()}\tExperiment:{experiment.click.sum()}")
print(f"Average click rate:\tControl:{control.click.mean()}\tExperiment:{experiment.click.mean()}")

Sample sizes:	Control: 3332 	Experiment: 2996
Total Clicks:	Control:932.0	Experiment:928.0
Average click rate:	Control:0.2797118847539016	Experiment:0.3097463284379172


In [101]:
import flatiron_stats as fs
fs.p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [103]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate*len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [105]:
import numpy as np
#Your code here
n = len(experiment)
p = control_rate
var = n*p*(1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [107]:
#Your code here

z_score = (experiment.click.sum()- expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


In [108]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score)#or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment:Yes, while the p-value is slightly loser,
        > both would lead to confident rejection of the null hypothesis.
        > The experimental page appears to be a more effective design. 
     

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.